# Itération 3 - Module BDD

In [ ]:
import sqlite3
import pandas as pd
GTFS = sqlite3.connect('../Ressources/gtfs_tag.db')

In [ ]:
requete = 'SELECT * FROM agency'
df = pd.read_sql(requete, GTFS)

In [ ]:
df

In [ ]:
from os import listdir

In [ ]:
path = '../Ressources/SEM-GTFS'
fich = listdir(path)
fich

In [ ]:
cursor = GTFS.cursor()

In [ ]:
for name in fich:
    pos = name.find(".")
    result = name[0:pos]
    print(result)

## Ecriture brute dans la base de donnée

In [ ]:
# trip 0
path2 = path + "/" + fich[0]
df= pd.read_csv(path2, sep=',')
df.head()
df2=df.loc[:, ["trip_id", "route_id", "service_id", "shape_id"]]
df2.set_index("trip_id")

In [ ]:
test3 = sqlite3.connect('../Ressources/test3.db')

In [ ]:
df2.to_sql('trips', con=test3, if_exists='append', index=False)

## Ecriture "manuel" dans la base de donnée

In [ ]:
#trip 0
path2 = path + "/" + fich[0]
df= pd.read_csv(path2, sep=',')
df.head()
df2=df.loc[:, ["trip_id", "route_id", "service_id", "shape_id"]]
df2

dico = {}
for j in df2.index:
    for i in df2.columns:
        dico[i] = df2.loc[j, i]

    cursor.execute("""INSERT INTO trips(trip_id, route_id, service_id, shape_id) 
    VALUES(:trip_id, :route_id, :service_id, :shape_id)""", dico)
GTFS.commit()

# agency 8
path2 = path + "/" + fich[8]
df= pd.read_csv(path2, sep=',')
df.head()
df2=df.loc[:, ["agency_id", "agency_name", "agency_url", "agency_timezone"]]
df2


dico = {}
for j in df2.index:
    for i in df2.columns:
        dico[i] = df2.loc[j, i]

    cursor.execute("""INSERT INTO agency(agency_id, agency_name, agency_url, agency_timezone) 
    VALUES(:agency_id, :agency_name, :agency_url, :agency_timezone)""", dico)
GTFS.commit()


# routes 9
path2 = path + "/" + fich[9]
df= pd.read_csv(path2, sep=',')
df.head()
df2=df.loc[:, ["route_id", "agency_id", "route_short_name", "route_long_name"]]
df2


dico = {}
for j in df2.index:
    for i in df2.columns:
        dico[i] = df2.loc[j, i]

    cursor.execute("""INSERT INTO routes(route_id, agency_id, route_short_name, route_long_name) 
    VALUES(:route_id, :agency_id, :route_short_name, :route_long_name)""", dico)
GTFS.commit()

#stop_times 11
path2 = path + "/" + fich[11]
df= pd.read_csv(path2, sep=',')
df.head()
df2=df.loc[:, ["trip_id", "arrival_time", "departure_time", "stop_id", "stop_sequence"]]
df2

dico = {}
for j in df2.index:
    for i in df2.columns:
        dico[i] = df2.loc[j, i]

    cursor.execute("""INSERT INTO stop_times(trip_id, arrival_time, departure_time, stop_id, stop_sequence) 
    VALUES(:trip_id, :arrival_time, :departure_time, :stop_id, :stop_sequence)""", dico)
GTFS.commit()

#stops 5
path2 = path + "/" + fich[5]
df= pd.read_csv(path2, sep=',')
df.head()
df2=df.loc[:, ["stop_id", "stop_name", "stop_lat", "stop_lon", "zone_id", "parent_station"]]
df2

dico = {}
for j in df2.index:
    for i in df2.columns:
        dico[i] = df2.loc[j, i]

    cursor.execute("""INSERT INTO stops(stop_id, stop_name, stop_lat, stop_lon, zone_id, parent_station) 
    VALUES(:stop_id, :stop_name, :stop_lat, :stop_lon, :zone_id, :parent_station)""", dico)
GTFS.commit()


## Ecriture "automatique" dans la base de donnée

In [ ]:
def gen_insert_query(table_name:str, a_dict:dict) -> str:
    result = "INSERT INTO " + table_name + "("
    for i, key in enumerate(a_dict):
        if i == len(a_dict) - 1:
            result += key +") "
        else:
            result += key +", "
    result += "VALUES("
    for i, key in enumerate(a_dict):
        if i == len(a_dict) - 1:
            result += ":" + key +")"
        else:
            result += ":" + key  + ", "

    return result

In [ ]:
def get_insert_queries(tablename:str, df: pd.DataFrame) -> list:
    result = []
    for j in df.index:
        dico = {}
        for i in df.columns:
            dico[i] = df.loc[j, i] 
#         print(dico)
        result.append(gen_insert_query(tablename, dico))
    return result


In [ ]:
def gen_insert_file(filename: str, tablename, df):
       
    # Remplissage de la base
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    liste_insert = get_insert_queries(tablename, df)
    for j, l in enumerate(liste_insert):
        dico = {i: df.loc[j, i] for i in df.columns}
#         print(dico)
        cursor.execute(l, dico)
    conn.commit()
    conn.close()
    
    

In [ ]:
# agency 8
path2 = path + "/" + fich[8]
df= pd.read_csv(path2, sep=',')
df.head()
df_agence=df.loc[:, ["agency_id", "agency_name", "agency_url", "agency_timezone"]]
df_agence.head(2)

In [ ]:
# routes 9
path2 = path + "/" + fich[9]
df= pd.read_csv(path2, sep=',')
df.head()
df_route=df.loc[:, ["route_id", "agency_id", "route_short_name", "route_long_name"]]
df_route.head(2)

In [ ]:
# Stop_time 11
path2 = path + "/" + fich[11]
df= pd.read_csv(path2, sep=',')
df.head()
df_stopt=df.loc[:, ["trip_id", "arrival_time", "departure_time", "stop_id", "stop_sequence"]]
df_stopt.head(2)

In [ ]:
# trips 0
path2 = path + "/" + fich[0]
df= pd.read_csv(path2, sep=',')
df.head()
df_trip=df.loc[:, ["trip_id", "route_id", "service_id", "shape_id"]]
df_trip.head(2)


In [ ]:
# Stops 5
path2 = path + "/" + fich[5]
df= pd.read_csv(path2, sep=',')
df.head()
df_stops=df.loc[:, ["stop_id", "stop_name", "stop_lat", "stop_lon", "zone_id", "parent_station"]]
df_stops.head(2)

In [ ]:
gen_insert_file('../Ressources/test2.db', 'agency', df_agence)
print('agence')
gen_insert_file('../Ressources/test2.db', 'routes', df_route)
print('route')
gen_insert_file('../Ressources/test2.db', 'stop_times', df_stopt)
print('stop_times')
gen_insert_file('../Ressources/test2.db', 'trips', df_trip)
print('trips')
gen_insert_file('../Ressources/test2.db', 'stops', df_stops)
print('stops')